In [8]:
dataset = pd.read_csv('./workdir/annotation_small.csv')
dataset[dataset['Катя (short list)'].isin({1.0, 0.0})].to_csv('./workdir/annotation_small_selected.csv')

In [12]:
import pandas as pd
from argparse import ArgumentParser
from pymystem3 import Mystem
from tqdm import tqdm
from gensim.models import FastText

morph = Mystem()

model_path = './workdir/models/model_fast2vec_300_5'
vectors_path = './workdir/vectors.txt'
#compounds_path = './workdir/annotation_katya_selected.csv'
compounds_path = './workdir/annotation_small_selected.csv'

comp = pd.read_csv(compounds_path)

chast1 = list(comp['Часть 1'].values)
chast2 = list(comp['Часть 2'].values)

print('loading model')
model = FastText.load(model_path)
print('loaded')
print(len(model.wv.vocab))

part1_lem = []
part2_lem = []

print('Starting lemmatize')

for w1, w2 in zip(chast1, chast2):
    lem_w1 = morph.lemmatize(w1)[0]
    lem_w2 = morph.lemmatize(w2)[0]
    part1_lem.append(lem_w1)
    part2_lem.append(lem_w2)

with open(vectors_path, 'w') as vectors:
    for w1, w2 in zip(part1_lem, part2_lem):
        if w1 in model.wv.vocab:
            vectors.write(w1 + ' ' + ' '.join(model.wv[w1].astype('str')) + '\r\n')
        else:
            print(w1)
        if w2 in model.wv.vocab:
            vectors.write(w2 + ' ' + ' '.join(model.wv[w2].astype('str')) + '\r\n')
        else:
            print(w2)

loading model
loaded
1060891
Starting lemmatize


In [13]:
import pandas as pd
from argparse import ArgumentParser
from pymystem3 import Mystem
from tqdm import tqdm
from gensim.models import FastText

morph = Mystem()

model_path = './workdir/models/model_fast2vec_300_5'
vecotr_path = './workdir/vectors_compounds.txt'
#compounds_path = './workdir/annotation_katya_selected.csv'
compounds_path = './workdir/annotation_small_selected.csv'


comp = pd.read_csv(compounds_path)

chast1 = list(comp['Часть 1'].values)
chast2 = list(comp['Часть 2'].values)

print('loading model')
model = FastText.load(model_path)
print('loaded')
print(len(model.wv.vocab))

comp_lem = []

print('Starting lemmatize')

for w1, w2 in zip(chast1, chast2):
    lem_w1 = morph.lemmatize(w1)[0]
    lem_w2 = morph.lemmatize(w2)[0]
    comp_lem.append(lem_w1 + '_' + lem_w2)

with open(vecotr_path, 'w') as vectors:
    for w1 in comp_lem:
        if w1 in model.wv.vocab:
            vectors.write(w1 + ' ' + ' '.join(model.wv[w1].astype('str')) + '\r\n')
        else:
            print(w1)

loading model
loaded
1060891
Starting lemmatize
приговор_судно
сектор_мишень
трагический_плач
красный_черт
неблагодарный_дело


In [14]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from scipy.spatial import distance
from pymystem3 import Mystem
from argparse import ArgumentParser


#compounds_path = './workdir/annotation_katya_selected.csv'
compounds_path = './workdir/annotation_small_selected.csv'

morph = Mystem()

def acquiring(wordvecs, compvecs):
    comp = pd.read_csv(compounds_path)

    part1 = list(comp['Часть 1'].values)
    part2 = list(comp['Часть 2'].values)
    values = list(comp['Катя (short list)'].values)
    
    print()

    compounds = []
    classes = []

    for w1, w2, v in zip(part1, part2, values):
        if v != 2:
            lem_w1 = morph.lemmatize(w1)[0]
            lem_w2 = morph.lemmatize(w2)[0]
            compounds.append('_'.join([lem_w1, lem_w2]))
            classes.append(v)

    vecs1 = []
    vecs2 = []
    vecsc = []
    true_comp_class = []

    words = []
    comps = []
    
    with open(wordvecs) as w:
        for line in w:
            words.append(line.split())

    with open(compvecs) as c:
        for line in c:
            comps.append(line.split())

    print(len(compounds))
    print(len(values))
    for compound, value in zip(compounds, values):
        comp_flag = 0
        w1_flag = 0
        w2_flag = 0
        for line in comps:
            if compound == line[0]:
                vecc = np.array(line[1:]).astype(np.float32)
                comp_flag = 1
        w1 = compound.split('_')[0]
        w2 = compound.split('_')[1]
        if comp_flag:
            for line in words:
                if w1 == line[0]:
                    vec1 = np.array(line[1:]).astype(np.float32)
                    w1_flag = 1
                    break
            for line in words:
                if w2 == line[0]:
                    w2_flag = 1
                    vec2 = np.array(line[1:]).astype(np.float32)
                    break
        if comp_flag and w1_flag and w2_flag:
            vecs1.append(vec1)
            vecs2.append(vec2)
            vecsc.append(vecc)
            true_comp_class.append(value)

    print(len(vecsc), 'examples retrieved for experiment')
    return vecs1, vecs2, vecsc, true_comp_class


def get_mean(part1_vecs, part2_vecs):
    parts_mean = []
    for vec1, vec2 in zip(part1_vecs, part2_vecs):
        parts_mean.append((vec1 + vec2) / 2)

    return parts_mean


def cosine_between_parts_and_compound(part1_vecs, part2_vecs, comp_vecs, true_class):
    parts_mean = get_mean(part1_vecs, part2_vecs)

    cosines = []
    for w, c in zip(parts_mean, comp_vecs):
        cosines.append(abs(1 - distance.cosine(w, c)))
    
    print(spearmanr(cosines, true_class)[0])


def chebyshev_between_parts_and_compound(part1_vecs, part2_vecs, comp_vecs, true_class):
    parts_mean = get_mean(part1_vecs, part2_vecs)

    chebyshevs = []
    for w, c in zip(parts_mean, comp_vecs):
        chebyshevs.append(abs(distance.chebyshev(w, c)))

    print(spearmanr(chebyshevs, true_class)[0])


def manhattan_between_parts_and_compound(part1_vecs, part2_vecs, comp_vecs, true_class):
    parts_mean= get_mean(part1_vecs, part2_vecs)

    manhattans = []
    for w, c in zip(parts_mean, comp_vecs):
        manhattans.append(abs(distance.cityblock(w, c)))
    print(spearmanr(manhattans, true_class)[0])


def euclidean_between_parts_and_compound(part1_vecs, part2_vecs, comp_vecs, true_class):
    parts_mean= get_mean(part1_vecs, part2_vecs)

    euclideans = []
    for w, c in zip(parts_mean, comp_vecs):
        euclideans.append(abs(distance.euclidean(w, c)))
    print(spearmanr(euclideans, true_class)[0])


vectors_parts_path = './workdir/vectors.txt'
vectors_compounds_path = './workdir/vectors_compounds.txt'

w1, w2, c, true = acquiring(vectors_parts_path, vectors_compounds_path)

cosine_between_parts_and_compound(w1, w2, c, true)
chebyshev_between_parts_and_compound(w1, w2, c, true)
manhattan_between_parts_and_compound(w1, w2, c, true)
euclidean_between_parts_and_compound(w1, w2, c, true)


201
201
196 examples retrieved for experiment
0.3952770772648245
-0.31842107787166385
-0.32484633827109205
-0.32188083347135593
